In [3]:
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize
import spacy
import networkx as nx
import matplotlib.pyplot as plt
from operator import add
import time
from scripts.detail_extractor import actionTaken, getViolations, getViolators, getAllPenalties, getDate, getViolations2, getViolations3

In [4]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [5]:
def identify_crime(text, types, folders):
    text = text.lower()
    data = []
    count = [0, 0, 0, 0]
    for i in range(len(types)):
        if(text.find(types[i]) != -1):
            data.append(folders[i])
            count[i] = 1
    return data, count

In [14]:
data = pd.read_csv('./results/data_large.csv')

types = ['insider', 'misappropriate', 'unregister', 'fraud']
folders = ['Insider Trading', 'Misappropriated Funds', 'Unregistered', 'Fraud']
count = [0, 0, 0, 0]
crimes = []
actions = []
violations = []
violators = []
fines = []
dates = []
years = []
texts = []

yrs = set()
counts = {}

for i in range(len(data['Text'])):
    text = data['Text'][i]
    text = text.lower()
    texts.append(text)
    crimes.append(identify_crime(text, types, folders))
    action = actionTaken(text)
    violation = getViolations3(text)
    violator = getViolators(text)
    fine = getAllPenalties(text)
    date, year = getDate(text)
    violations.append(violation)
    violators.append(violator)
    actions.append(action)
    fines.append(fine)
    dates.append(date)
    years.append(year)
    if(year != ''):
        yrs.add(year)
        
y = sorted(yrs)
        
for i in range(len(y)):
    counts[y[i]] = count
    
total = [0, 0, 0, 0]
    
for i in range(len(data['Text'])):
    text = data['Text'][i]
    _, count = identify_crime(text, types, folders)
    _, year = getDate(text)
    if(year != ''):
        counts[year] = list(map(add, counts[year], count))
    

data['Text'] = texts
data['Crimes'] = crimes
data['Violations'] = violations
data['Violators'] = violators
data['Action Taken'] = actions
data['Fines'] = fines
data['Dates'] = dates
data['Years'] = years

In [15]:
data.head(40)

,Text,Crimes,Violations,Violators,Action Taken,Fines,Dates,Years
0,the securities and exchange commission has cha...,"([Misappropriated Funds, Fraud], [0, 1, 0, 1])","[section 17(a) of the securities act of 1933, ...",[],[],[],,
1,the securities and exchange commission today a...,"([Misappropriated Funds, Fraud], [0, 1, 0, 1])","[section 17(a) of the securities act, section ...",[],[the court granted the sec's requests for an a...,[],"december 6, 2018",2018
2,the securities and exchange commission today a...,"([Unregistered, Fraud], [0, 0, 1, 1])","[section 17(a) of the securities act of 1933, ...",[],"[the settlement is subject to court approval, ...",[],"june 5, 2018",2018
3,the securities and exchange commission charged...,"([Misappropriated Funds], [0, 1, 0, 0])","[sections 206(1), 206(2), and 206(4) of the ad...",[],"[without admitting or denying the allegations,...",[],,
4,"on november 19, 2018, a u.s. district court fo...","([Misappropriated Funds, Fraud], [0, 1, 0, 1])",[section 10(b) of the securities exchange act ...,[],[without admitting or denying the allegations ...,"[[Disgorgement, $24,655,000], [Prejudgment int...","november 19, 2018",2018
5,the securities and exchange commission today a...,"([Fraud], [0, 0, 0, 1])","[section 17(a) of the securities act of 1933, ...",[],[without admitting or denying the sec's allega...,[],,
6,the securities and exchange commission today c...,"([Unregistered, Fraud], [0, 0, 1, 1])","[section 17(a) of the securities act of 1933, ...",[],"[the complaint seeks a permanent injunction, d...",[],,
7,the securities and exchange commission charged...,"([Misappropriated Funds, Fraud], [0, 1, 0, 1])","[section 17(a) of the securities act of 1933, ...",[],[without admitting or denying the allegations ...,[],,
8,the securities and exchange commission charged...,"([Fraud], [0, 0, 0, 1])",[sections 17(a)(1) and (3) of the securities a...,[],[],[],"october 2, 2018",2018
9,the securities and exchange commission yesterd...,"([Fraud], [0, 0, 0, 1])","[section 17(a) of the securities act of 1933, ...",[],"[colucci and noska each agreed to a $25,000 pe...","[[Penalty, $15,000], [Penalty, $15,000]]",,


In [38]:
new = pd.DataFrame.from_dict(counts)
new['Crime'] = ['Insider Trading', 'Misappropriated Funds', 'Unregistered', 'Fraud']
cols = new.columns.tolist()
cols = cols[-1:] + cols[:-1]
new = new[cols]
new.to_csv('results/metadata_large.csv', index=False)

In [39]:
new.head()

,Crime,1114,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Insider Trading,0,15,1,24,31,26,40,34,32,...,43,35,6,7,8,5,15,16,9,7
1,Misappropriated Funds,0,16,0,12,18,11,15,29,47,...,30,19,3,4,6,7,9,27,12,11
2,Unregistered,0,24,0,18,30,15,32,29,48,...,36,25,3,4,7,4,10,33,13,12
3,Fraud,1,82,9,91,122,93,168,152,201,...,145,90,6,17,22,23,51,101,81,53


In [13]:
data.to_csv('./results/data_large.csv', index=False)